# Modeling Geographic Data to Gold Layer

## Environment Configuration

In [0]:
from src.utils.udfs import functions_for_df_structure_management as ffdsm

## Data Ingestion from Silver Layer

In [0]:
df_silver_state_and_capital_of_declarants_residence = spark.table("brazilian_tax_big_numbers.silver_layer.delta_estados_e_capitais")

In [0]:
display(df_silver_state_and_capital_of_declarants_residence)

In [0]:
df_silver_municipality_form_type = spark.table("brazilian_tax_big_numbers.silver_layer.delta_municipio_e_tipo_de_declaracao")

In [0]:
display(df_silver_municipality_form_type)

## Data Modeling

In [0]:
df_joined = df_silver_state_and_capital_of_declarants_residence.join(
    df_silver_municipality_form_type,
    on=["AnoCalendario", "EstadoSigla"],
    how="full_outer"
)

In [0]:
df_filtered = df_joined.filter(df_joined.AnoCalendario >= 2016)

In [0]:
df_filtered = df_filtered.drop("QuantidadeDeDeclarantesEstadoDeResidenciaDoDeclarante")

In [0]:
df_filtered.count()

In [0]:
df_nulls = ffdsm.count_nulls(df_filtered)
display(df_nulls)

## Save Table as Delta in Gold Layer

In [0]:
spark.sql(f"CREATE SCHEMA IF NOT EXISTS brazilian_tax_big_numbers.gold_layer")

In [0]:
error = None

try:
    df_filtered.write \
        .mode("overwrite") \
        .saveAsTable(f"brazilian_tax_big_numbers.gold_layer.delta_dados_geograficos")
    error = None
except Exception as e:
    error = str(e)
    print(error)